In [1]:
import os 

In [2]:
%pwd

'C:\\Users\\Param\\Python-Project\\CampusX\\GitHub\\end_to_end-ML_Project_with_MLFLOW\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\Param\\Python-Project\\CampusX\\GitHub\\end_to_end-ML_Project_with_MLFLOW'

## Define your Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str 
    alpha: float
    l1_ratio: float
    target_column: str 

In [12]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config_filepath = read_yaml(config_filepath)
        self.params_filepath = read_yaml(params_file_path)
        self.schema_filepath = read_yaml(schema_file_path)

        create_directories([self.config_filepath.artifacts_root])
        
    def get_model_config(self) -> ModelConfig:
        config = self.config_filepath.model_trainer
        params = self.params_filepath.ElasticNet
        schema = self.schema_filepath.TARGET_COLUMNS 
        
        create_directories([config.root_dir])
        
        model_config = ModelConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            alpha= params.alpha,
            l1_ratio= params.l1_ratio,
            target_column=schema.name
        )
        
        return model_config

In [13]:
import pandas as pd
import os 
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

class ModelTrainer:
    def __init__(self, config: ModelConfig):
        self.config = config
        
    def train_model(self):
        train_data =  pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x =  train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        

In [14]:
try:
    config_manager = ConfigurationManager()
    model_config = config_manager.get_model_config()
    model_trainer = ModelTrainer(config=model_config)
    model_trainer.train_model()
except Exception as e:
    raise e 

[2023-12-21 00:40:30,715: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-21 00:40:30,717: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-21 00:40:30,718: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-21 00:40:30,719: INFO: common: created directory at: artifacts]
[2023-12-21 00:40:30,720: INFO: common: created directory at: artifacts/model_trainer]
